Monte Carlo Tree Search (MCTS) without neural networks:

   This is a search algorithm that can be used to create strong chess engines without relying on deep learning. It involves simulating many possible future game states and choosing moves based on these simulations.

In [1]:
import os
import sys
src_path = os.path.abspath(os.path.join(os.path.dirname('__file__'), '..', 'src'))
sys.path.append(src_path)
import chess
import json
import chess
import math
from stockfish import Stockfish
import chess.svg
from IPython.display import SVG, display, clear_output
from chess_zero.config import Config, PlayWithHumanConfig
from chess_zero.agent.model_chess import ChessModel
from chess_zero.agent.player_chess import ChessPlayer
import numpy as np
import random
from chess_zero.env.chess_env import ChessEnv
from chess_zero.agent.model_chess import ChessModel
from chess_zero.lib.model_helper import load_best_model_weight

2024-09-25 23:03:27.980716: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-25 23:03:27.992406: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-25 23:03:27.996790: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-25 23:03:28.006203: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-25 23:03:28.684003: W tensorflow/compiler/tf2

In [2]:
import time
class Node:
    def __init__(self, state, parent=None, move=None):
        self.state = state
        self.parent = parent
        self.move = move
        self.children = []
        self.visits = 0
        self.value = 0

class MCTS:
    def __init__(self, root_state, exploration_weight=1.4):
        self.root = Node(root_state)
        self.exploration_weight = exploration_weight

    def select(self, node):
        while node.children:
            if not all(child.visits > 0 for child in node.children):
                return self.expand(node)
            node = self.uct_select(node)
        return node

    def expand(self, node):
        for move in node.state.legal_moves:
            new_state = node.state.copy()
            new_state.push(move)
            child = Node(new_state, parent=node, move=move)
            node.children.append(child)
        return random.choice(node.children)

    def simulate(self, node):
        state = node.state.copy()
        while not state.is_game_over():
            move = random.choice(list(state.legal_moves))
            state.push(move)
        return self.get_result(state)

    def backpropagate(self, node, result):
        while node is not None:
            node.visits += 1
            node.value += result
            node = node.parent

    def uct_select(self, node):
        return max(node.children, key=lambda c: self.uct_value(c))

    def uct_value(self, node):
        if node.visits == 0:
            return float('inf')
        return (node.value / node.visits) + self.exploration_weight * math.sqrt(
            math.log(node.parent.visits) / node.visits)

    def get_result(self, state):
        if state.is_checkmate():
            return 1 if state.turn == chess.BLACK else -1
        elif state.is_stalemate() or state.is_insufficient_material() or state.is_seventyfive_moves() or state.is_fivefold_repetition():
            return 0
        else:
            return 0  # This shouldn't happen in a completed game

    def search(self, num_simulations):
        legal_moves = list(self.root.state.legal_moves)
        for move in legal_moves:
            self.expand_move(self.root, move)
        
        for _ in range(max(num_simulations - len(legal_moves), 0)):
            leaf = self.select(self.root)
            result = self.simulate(leaf)
            self.backpropagate(leaf, result)

    def expand_move(self, node, move):
        new_state = node.state.copy()
        new_state.push(move)
        child = Node(new_state, parent=node, move=move)
        node.children.append(child)
        result = self.simulate(child)
        self.backpropagate(child, result)

    def best_move(self):
        if not self.root.children:
            return random.choice(list(self.root.state.legal_moves))
        return max(self.root.children, key=lambda c: c.visits).move

def play_game():
    board = chess.Board()
    mcts = MCTS(board)

    while not board.is_game_over():
        if board.turn == chess.WHITE:
            start_time = time.time()
            while time.time() - start_time < 5:  # 5 seconds time limit
                mcts.search(100)  # Run in smaller batches
            move = mcts.best_move()
            if move is None:
                print("MCTS couldn't find a move. Resigning.")
                break
        else:
            # play against stockfish third best move
            stockfish = Stockfish(path="/usr/games/stockfish")
            stockfish.set_fen_position(board.fen())
            moves = stockfish.get_top_moves(3)
            print(f"Top 3 moves: {moves}")
            if not moves:
                print("Stockfish couldn't find a move. Resigning.")
                break
            move = chess.Move.from_uci(moves[2]['Move'])
        
        print(f"{'White' if board.turn == chess.WHITE else 'Black'} plays: {move}")
        board.push(move)
        print(board)
        print("\n")

        # Update MCTS root for the next turn
        if not board.is_game_over():
            if mcts.root.children:
                mcts.root = next((child for child in mcts.root.children if child.move == move), None)
                if mcts.root is None:
                    mcts.root = Node(board)
                else:
                    mcts.root.parent = None
            else:
                mcts.root = Node(board)

    print("Game Over")
    print("Result:", board.result())

if __name__ == "__main__":
    play_game()

White plays: e2e4
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R


Top 3 moves: [{'Move': 'c7c5', 'Centipawn': 34, 'Mate': None}, {'Move': 'c7c6', 'Centipawn': 46, 'Mate': None}, {'Move': 'e7e6', 'Centipawn': 51, 'Mate': None}]
Black plays: e7e6
r n b q k b n r
p p p p . p p p
. . . . p . . .
. . . . . . . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R


White plays: d1g4
r n b q k b n r
p p p p . p p p
. . . . p . . .
. . . . . . . .
. . . . P . Q .
. . . . . . . .
P P P P . P P P
R N B . K B N R


Top 3 moves: [{'Move': 'd7d5', 'Centipawn': -62, 'Mate': None}, {'Move': 'b8c6', 'Centipawn': -53, 'Mate': None}, {'Move': 'c7c5', 'Centipawn': -50, 'Mate': None}]
Black plays: c7c5
r n b q k b n r
p p . p . p p p
. . . . p . . .
. . p . . . . .
. . . . P . Q .
. . . . . . . .
P P P P . P P P
R N B . K B N R


White plays: g4g7
r n b q k b n r
p p . p . p Q p
. . . . p . . .
. . p . . . . .
.